In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image
import numpy as np
from scipy.fftpack import dct, idct
import matplotlib.pyplot as plt

# DCT ve IDCT işlemleri
def dct2(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')

def idct2(block):
    return idct(idct(block.T, norm='ortho').T, norm='ortho')

# Mesajı gömme fonksiyonu
def embed_message_dct(image, message):
    img = image.convert('L')  # Gri tonlamaya çevir
    arr = np.array(img, dtype=np.float32)

    h, w = arr.shape
    max_chars = (h // 8) * (w // 8)

    if len(message) > max_chars:
        raise ValueError(f"Mesaj çok uzun! Maksimum {max_chars} karakter gömülebilir.")

    char_index = 0
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            if char_index >= len(message):
                break

            char = message[char_index]
            binary_char = format(ord(char), '08b')
            block = arr[i:i+8, j:j+8]
            dct_block = dct2(block)

            for k in range(8):
                val = int(dct_block[1, k])
                val = (val & ~1) | int(binary_char[k])
                dct_block[1, k] = val

            arr[i:i+8, j:j+8] = idct2(dct_block)
            char_index += 1

    new_img = Image.fromarray(np.clip(arr, 0, 255).astype(np.uint8))
    return new_img

# Mesajı çıkarma fonksiyonu
def extract_message_dct(image, char_count):
    img = image.convert('L')
    arr = np.array(img, dtype=np.float32)

    h, w = arr.shape
    chars = []

    char_index = 0
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            if char_index >= char_count:
                break

            block = arr[i:i+8, j:j+8]
            dct_block = dct2(block)

            bits = ''
            for k in range(8):
                bits += str(int(dct_block[1, k]) & 1)

            chars.append(chr(int(bits, 2)))
            char_index += 1

    return ''.join(chars)

# ✅ Kullanıcıdan mesaj al
message = input("Gizlemek istediğiniz mesajı girin (en fazla 160 karakter): ")
if len(message) > 160:
    raise ValueError("Maksimum 160 karakter girebilirsiniz!")

# ✅ Resmi yükle (gri tonlamalı ve yeterli büyük boyutta olmalı)
image_path = "/kaggle/input/cifar10-pngs-in-folders/cifar10/test/airplane/0002.png"
image = Image.open(image_path)

# ✅ Mesajı göm
embedded = embed_message_dct(image, message)

# ✅ Göster (isteğe bağlı)
plt.imshow(embedded, cmap='gray')
plt.axis('off')
plt.title("Mesaj Gömülmüş Görsel")
plt.show()

# ✅ Mesajı çöz
decoded = extract_message_dct(embedded, len(message))
print("Çıkarılan mesaj:", decoded)
